In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path
from neo4j import GraphDatabase

In [2]:
load_dotenv(dotenv_path=Path("../.env"))

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = os.getenv("NEO4J_URI")
AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()